## Police Violence Data 

notebook for adding geolocation data to the Police Brutality 2020 database

In [ ]:
# imports
import sys
import pandas as pd
import numpy as np
from google.colab import files

### PB 2020 Data

[police violence incident repo](https://github.com/2020PB/police-brutality)


In [ ]:
# police brutality data
pb_csv = pd.read_csv('https://raw.githubusercontent.com/2020PB/police-brutality/data_build/all-locations.csv')
pb_df = pd.DataFrame(pb_csv)

In [ ]:
print(pb_df.shape)
pb_df.info()

(1022, 27)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 27 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   state      1022 non-null   object
 1   edit_at    1022 non-null   object
 2   city       1021 non-null   object
 3   name       1022 non-null   object
 4   date       1018 non-null   object
 5   date_text  1022 non-null   object
 6   id         1022 non-null   object
 7   Link 1     1022 non-null   object
 8   Link 2     589 non-null    object
 9   Link 3     336 non-null    object
 10  Link 4     199 non-null    object
 11  Link 5     119 non-null    object
 12  Link 6     79 non-null     object
 13  Link 7     47 non-null     object
 14  Link 8     41 non-null     object
 15  Link 9     26 non-null     object
 16  Link 10    21 non-null     object
 17  Link 11    18 non-null     object
 18  Link 12    13 non-null     object
 19  Link 13    9 non-null      object
 20  Link 14    8 non-nu

In [ ]:
def clean_pb2020(df):
  # rename pb_df columns
  df.rename(columns={'state':'STATE_NAME', 'city':'CITY'}, inplace=True)  
  # change substandard city and state names
  df['CITY']= df['CITY'].str.replace('New York City', 'New York', case = False)
  df['CITY']= df['CITY'].str.replace('DC', 'Washington', case = True)
  df['CITY']= df['CITY'].str.replace('Hollywood', 'Los Angeles', case = True)
  df['STATE_NAME']= df['STATE_NAME'].str.replace('Washington DC', 'District of Columbia', case = False)
  # drop NaNs
  df.dropna(subset = ['CITY', 'date'], inplace=True)
  # put date column in datetime
  df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
  # remove whitespace from columns
  df['CITY'] = df['CITY'].str.strip()
  df['STATE_NAME'] = df['STATE_NAME'].str.strip()
  return df

In [ ]:
pb_df = clean_pb2020(pb_df)

In [ ]:
print(pb_df.shape)
pb_df.isnull().sum()

(1017, 27)


STATE_NAME       0
edit_at          0
CITY             0
name             0
date             0
date_text        0
id               0
Link 1           0
Link 2         429
Link 3         681
Link 4         818
Link 5         898
Link 6         938
Link 7         970
Link 8         976
Link 9         991
Link 10        996
Link 11        999
Link 12       1004
Link 13       1008
Link 14       1009
Link 15       1011
Link 16       1013
Link 17       1014
Link 18       1014
Link 19       1014
Link 20       1014
dtype: int64

### Location Data

[US geolocations repo](https://github.com/kelvins/US-Cities-Database)

In [ ]:
# location data
loc_csv = pd.read_csv('https://raw.githubusercontent.com/kelvins/US-Cities-Database/master/csv/us_cities.csv')
loc_df = pd.DataFrame(loc_csv)

In [ ]:
print(loc_df.shape)
loc_df.info()

(29880, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29880 entries, 0 to 29879
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          29880 non-null  int64  
 1   STATE_CODE  29880 non-null  object 
 2   STATE_NAME  29880 non-null  object 
 3   CITY        29880 non-null  object 
 4   COUNTY      29858 non-null  object 
 5   LATITUDE    29880 non-null  float64
 6   LONGITUDE   29880 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.6+ MB


In [ ]:
loc_df.isna().sum()

ID             0
STATE_CODE     0
STATE_NAME     0
CITY           0
COUNTY        22
LATITUDE       0
LONGITUDE      0
dtype: int64

In [ ]:
loc_df.dtypes

ID              int64
STATE_CODE     object
STATE_NAME     object
CITY           object
COUNTY         object
LATITUDE      float64
LONGITUDE     float64
dtype: object

In [ ]:
def clean_loc(df):
  # drop redundant id column in loc_df
  df = df.drop('ID', axis=1)
  # drop rows with the same city and state but different counties
  df = df.drop_duplicates(subset=['STATE_CODE','CITY'], keep='first')
  # add missing cities
  Ferguson = {'STATE_CODE':'MO' ,'STATE_NAME':'Missouri', 'CITY':'Ferguson','COUNTY':'St. Louis','LATITUDE':38.744167,'LONGITUDE':-90.305278}
  DC = {'STATE_CODE':'DC' ,'STATE_NAME':'District of Columbia', 'CITY':'Washington','COUNTY':'St. Louis','LATITUDE':38.912217,'LONGITUDE':-77.017691}
  df = df.append(Ferguson, ignore_index=True)
  df = df.append(DC, ignore_index=True)
  return df

In [ ]:
loc_df = clean_loc(loc_df)

In [ ]:
print(loc_df.shape)
loc_df.head()

(29740, 6)


,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE
0,AK,Alaska,Adak,Aleutians West,55.999722,-161.207778
1,AK,Alaska,Akiachak,Bethel,60.891854,-161.392330
2,AK,Alaska,Akiak,Bethel,60.890632,-161.199325
3,AK,Alaska,Akutan,Aleutians East,54.143012,-165.785368
4,AK,Alaska,Alakanuk,Wade Hampton,62.746967,-164.602280


In [ ]:
print(loc_df.shape)
loc_df.head()

(29740, 6)


,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE
0,AK,Alaska,Adak,Aleutians West,55.999722,-161.207778
1,AK,Alaska,Akiachak,Bethel,60.891854,-161.392330
2,AK,Alaska,Akiak,Bethel,60.890632,-161.199325
3,AK,Alaska,Akutan,Aleutians East,54.143012,-165.785368
4,AK,Alaska,Alakanuk,Wade Hampton,62.746967,-164.602280


### Merge and Save

In [ ]:
# merge our two dfs
# incident_df = pd.merge(pb_df, loc_df, how='left')
incident_df = pb_df.merge(loc_df, how='inner')

In [ ]:
print(incident_df.shape)
incident_df.tail()

(1041, 31)


,STATE_NAME,edit_at,CITY,name,date,date_text,id,Link 1,Link 2,Link 3,Link 4,Link 5,Link 6,Link 7,Link 8,Link 9,Link 10,Link 11,Link 12,Link 13,Link 14,Link 15,Link 16,Link 17,Link 18,Link 19,Link 20,STATE_CODE,COUNTY,LATITUDE,LONGITUDE
1036,Nebraska,https://github.com/2020PB/police-brutality/blo...,Lincoln,Reporter shows tear gas canister fired at him ...,2020-05-31,May 31st,ne-lincoln-2,https://twitter.com/ChrisDunkerLJS/status/1268...,https://twitter.com/ChrisDunkerLJS/status/1268...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,Lancaster,40.8169,-96.7103
1037,Nebraska,https://github.com/2020PB/police-brutality/blo...,Lincoln,Woman bleeding from face after being shot by p...,2020-05-31,May 31st,ne-lincoln-3,https://www.youtube.com/watch?v=GhZieEDPfv4,https://www.facebook.com/LincolnJournalStar/vi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,Lancaster,40.8169,-96.7103
1038,Nebraska,https://github.com/2020PB/police-brutality/blo...,Omaha,"Police Mace, shoot pepper bullets at protester...",2020-05-31,May 31st,ne-omaha-1,https://twitter.com/XruthxNthr/status/12669032...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,Douglas,41.2610,-95.9376
1039,Nebraska,https://github.com/2020PB/police-brutality/blo...,Omaha,Peaceful protesters arrested for breaking curfew,2020-06-01,June 1,ne-omaha-2,https://twitter.com/reecereports/status/126762...,https://twitter.com/greg_doucette/status/12676...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,Douglas,41.2610,-95.9376
1040,Nebraska,https://github.com/2020PB/police-brutality/blo...,Omaha,Police officer shoves protester on bike; polic...,2020-07-25,July 25th,ne-omaha-3,https://twitter.com/greg_doucette/status/12874...,https://twitter.com/greg_doucette/status/12874...,https://twitter.com/greg_doucette/status/12874...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,Douglas,41.2610,-95.9376


In [ ]:
# double check for NaNs
incident_df.isnull().sum()

STATE_NAME       0
edit_at          0
CITY             0
name             0
date             0
date_text        0
id               0
Link 1           0
Link 2         440
Link 3         697
Link 4         838
Link 5         919
Link 6         960
Link 7         994
Link 8         999
Link 9        1015
Link 10       1020
Link 11       1023
Link 12       1028
Link 13       1032
Link 14       1033
Link 15       1035
Link 16       1037
Link 17       1038
Link 18       1038
Link 19       1038
Link 20       1038
STATE_CODE       0
COUNTY          20
LATITUDE         0
LONGITUDE        0
dtype: int64

In [ ]:
# saving the dataframe 
incident_df.to_csv('pv_incidents.csv')
files.download('pv_incidents.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Data Exploration

In [ ]:
import pandas_profiling

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pip uninstall -y pandas_profiling

Uninstalling pandas-profiling-2.9.0rc1:
  Successfully uninstalled pandas-profiling-2.9.0rc1


In [ ]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
  Created wheel for pandas-profiling: filename=pandas_profiling-2.9.0rc1-py2.py3-none-any.whl size=258106 sha256=83abdb0689903a9d09ff8676b68cd1a46c1fd6b4d3d554324257e7617ec8338a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ianmzcy1/wheels/56/c2/dd/8d945b0443c35df7d5f62fa9e9ae105a2d8b286302b92e0109
Successfully built pandas-profiling


In [ ]:
# Check Pandas Profiling version
from pandas_profiling import ProfileReport

pandas_profiling.__version__

'2.9.0rc1'

In [ ]:
profile = ProfileReport(incident_df)
profile